In [12]:
import json
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:

# Load data
with open('daftar_hotel_di_kota_bandung.json') as f:
    data = json.load(f)

In [3]:
# Ekstraksi fitur
features = ['kategori_hotel', 'klasifikasi_hotel', 'rating_hotel', 'fasilitas']


In [4]:
# Gabung fitur-fitur teks menjadi satu teks
def combine_features(row):
    return ' '.join([str(row[feature]) for feature in features])


In [5]:
# Buat matriks fitur
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform([combine_features(row) for row in data])


In [6]:
# Fungsi untuk merekomendasikan hotel
def recommend_hotels(user_preference, num_recommendations=3):
    # Ekstraksi fitur dari preferensi pengguna
    user_feature_matrix = tfidf.transform([user_preference])

    # Hitung kemiripan antara preferensi pengguna dengan setiap hotel
    similarities = cosine_similarity(user_feature_matrix, feature_matrix)

    # Urutkan indeks hotel berdasarkan kemiripan
    hotel_indices = similarities.argsort()[0][::-1][:num_recommendations]

    # Ambil data hotel yang direkomendasikan
    recommended_hotels = [data[idx] for idx in hotel_indices]

    return recommended_hotels

In [14]:
# Interaksi dengan pengguna
def main():
    print("Selamat datang di layanan rekomendasi hotel!")
    user_input = input("Silakan berikan preferensi hotel Anda dalam satu kalimat: ")

    # Tokenisasi kalimat pengguna
    tokens = word_tokenize(user_input.lower())

    # Identifikasi preferensi pengguna
    user_preference = {
        'kategori_hotel': ' '.join([token for token in tokens if token in ['bintang', 'bintangnya']]),
        'klasifikasi_hotel': ' '.join([token for token in tokens if token in ['bisnis', 'liburan']]),
        'rating_hotel': 0,
        'fasilitas': ' '.join([token for token in tokens if token not in ['bintang', 'bintangnya', 'bisnis', 'liburan']])
    }

    recommended_hotels = recommend_hotels(user_input)

    print("\nBerikut adalah rekomendasi hotel untuk Anda:")
    for idx, hotel in enumerate(recommended_hotels, 1):
        print(f"{idx}. {hotel['nama_hotel']} ({hotel['kategori_hotel']}, {hotel['klasifikasi_hotel']})")
        print(f"   Rating: {hotel['rating_hotel']}")
        print(f"   Alamat: {hotel['alamat']}")
        print(f"   Harga: Rp {hotel['range_harga']['min']:,} - Rp {hotel['range_harga']['max']:,}")
        print(f"   Fasilitas: {', '.join(hotel['fasilitas'])}")
        print()

if __name__ == "__main__":
    main()

Selamat datang di layanan rekomendasi hotel!
Silakan berikan preferensi hotel Anda dalam satu kalimat: klasifikasi hotel 

Berikut adalah rekomendasi hotel untuk Anda:
1. NEW-B HOTEL (BINTANG 2, HOTEL)
   Rating: 3.8
   Alamat: JL. SUPRATMAN NO.23 - CIHAPIT - KEC. BANDUNG WETAN - KOTA BANDUNG - JAWA
  BARAT 40122
   Harga: Rp 199,075 - Rp 150,313
   Fasilitas: Free Wifi, AC, Televisi

2. DPALMA HOTEL (BINTANG 2, HOTEL)
   Rating: 4.1
   Alamat: JL. GATOT SUBROTO NO.41 - MALABAR - KEC. LENGKONG - KOTA BANDUNG - JAWA
  BARAT 40262
   Harga: Rp 161,456 - Rp 122,670
   Fasilitas: Free Wifi, AC, Televisi

3. IBIS BUDGET BANDUNG ASIA AFRIKA (BINTANG 2, HOTEL)
   Rating: 4.9
   Alamat: JL. ASIA AFRIKA NO.128 - PALEDANG - KEC. LENGKONG - KOTA BANDUNG - JAWA
  BARAT 40261
   Harga: Rp 104,478 - Rp 181,919
   Fasilitas: Free Wifi, AC, Televisi

